In [1]:
import os
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data import Dataset, DataLoader
import torchvision
import string
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from timeit import default_timer as timer


import ipynb.fs.full.MyDataset as MyDataset
import ipynb.fs.full.Models as Models
import ipynb.fs.full.Utils as Utils
import ipynb.fs.full.RMSELoss as RMSELoss
  

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        #transforms.Resize(size=400),
        #transforms.RandomRotation(degrees=2),
        #transforms.ColorJitter(),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

trans = image_transforms["train"]

In [3]:
pre_resized=True
if pre_resized:
    csv_file = "data/Ch2_002_resized/interpolated.csv"
    directory = "data/Ch2_002_resized"
else:
    csv_file = "data/Ch2_002/interpolated.csv"
    directory = "data/Ch2_002"  

dataset = MyDataset.MyDataset(csv_file, directory, transform = trans, enable_cache=False, reduce_near_zero_samples=False)


a,b,c = int(0.5*len(dataset)),int(0.25*len(dataset)),int(0.25*len(dataset))
m = len(dataset) - (a+b+c)



train_set, val_set, test_set = torch.utils.data.random_split(dataset, [a+m,b,c])
testloader = DataLoader(dataset, batch_size=1, shuffle=False, )


In [4]:
model = Models.ConvNetCustom()
model_data_file = "save_model/cusyom.pt"
model.load_state_dict(torch.load(model_data_file))

use_cuda = torch.cuda.is_available()
print("Gpu:",use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")

 
if use_cuda:
    model.cuda()
    model.to(device)
    
model.eval()

Gpu: True


ConvNetCustom(
  (conv1_p): Conv2d(3, 12, kernel_size=(12, 12), stride=(1, 1), padding=(5, 5))
  (bnm1_p): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_p): Conv2d(12, 32, kernel_size=(8, 8), stride=(1, 1), padding=(2, 2))
  (bnm2_p): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_p): Conv2d(32, 64, kernel_size=(6, 6), stride=(1, 1), padding=(2, 2))
  (bnm3_p): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1_p): Linear(in_features=6144, out_features=512, bias=True)
  (dropout1_p): Dropout(p=0.6, inplace=False)
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(3, 3))
  (bnm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 8, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (bnm2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_

In [5]:
criterion = RMSELoss.RMSELoss()

In [7]:
def to_angle(a):
    return (round(a * 57.2957795131, 3))

print("Start computing images")
i=0       
for data in testloader:


    inputs = data['X']
    angles = data['y']
    path = data['Path'][0]
    
    path_ = path.replace("_resized", "")
    
    name = path.split("/")[-1]
    save_path = str('new_test/')+str(name)
    


    
    inputs = inputs.to(device)
    angles = angles.to(device)
    outputs = torch.squeeze(model(inputs))
    
    
    loss = criterion(outputs, angles)
    
    if "center" in path:

        show=False
        i = i+1
        if i%250==0:
            print (i, save_path)
            show=True
        
        Utils.draw_angles_image(path_,save_path, -to_angle(angles.item()), -to_angle(outputs.item()), show=False, save=True )
        show=False

    #print(angles.item())



Start computing images
250 new_test/1479424228332837030.jpg
500 new_test/1479424240835023706.jpg
750 new_test/1479424253337227977.jpg
1000 new_test/1479424265839412576.jpg
1250 new_test/1479424278341629366.jpg
1500 new_test/1479424290843734525.jpg
1750 new_test/1479424303345937280.jpg
2000 new_test/1479424315848242574.jpg


KeyboardInterrupt: 

In [10]:
Utils.video_from_frames("new_test", "custom.avi")